# Tuning and Optimizing Neural Networks - Lab

## Introduction

Now that you've practiced regularization, initialization, and optimization techniques, its time to synthesize these concepts into a cohesive modeling pipeline.  

With this pipeline, you will not only fit an initial model but also attempt to improve it. Your final model selection will pertain to the test metrics across these models. This will more naturally simulate a problem you might be faced with in practice, and the various modeling decisions you are apt to encounter along the way.  

Recall that our end objective is to achieve a balance between overfitting and underfitting. You've seen the bias variance trade-off, and the role of regularization in order to reduce overfitting on training data and improving generalization to new cases. Common frameworks for such a procedure include train/validate/test methodology when data is plentiful, and K-folds cross-validation for smaller, more limited datasets. In this lab, you'll perform the latter, as the dataset in question is fairly limited. 

## Objectives

You will be able to:

* Apply normalization as a preprocessing technique 
* Implement a K-folds cross validation modeling pipeline for deep learning models 
* Apply regularization techniques to improve your model's performance 

## Load the data

First, run the following cell to import all the neccessary libraries and classes you will need in this lab. 

In [1]:
# Necessary libraries and classes
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_predict
from keras import models
from keras import layers
from keras import regularizers
from keras.wrappers.scikit_learn import KerasRegressor
from multiprocessing import Queue

Using TensorFlow backend.
/opt/conda/envs/learn-env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/envs/learn-env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/envs/learn-env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/envs/learn-env/lib/python3.6/site-pa

In this lab you'll be working with the *The Lending Club* data. 

- Import the data available in the file `'loan_final.csv'` 
- Drop rows with missing values in the `'total_pymnt'` column (this is your target column) 
- Print the first five rows of the data 
- Print the dimensions of the data 

In [2]:
# Import the data
data = pd.read_csv('loan_final.csv')

# Drop rows with no target value
data.dropna(subset=['total_pymnt'], inplace=True)

# Print the first five rows
data.head(2)

,loan_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,addr_state,total_acc,total_pymnt,application_type
0,5000.0,4975.0,36 months,10.65%,162.87,B,10+ years,RENT,24000.0,Verified,Fully Paid,credit_card,AZ,9.0,5863.155187,Individual
1,2500.0,2500.0,60 months,15.27%,59.83,C,< 1 year,RENT,30000.0,Source Verified,Charged Off,car,GA,4.0,1014.530000,Individual


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42535 entries, 0 to 42537
Data columns (total 16 columns):
loan_amnt              42535 non-null float64
funded_amnt_inv        42535 non-null float64
term                   42535 non-null object
int_rate               42535 non-null object
installment            42535 non-null float64
grade                  42535 non-null object
emp_length             41423 non-null object
home_ownership         42535 non-null object
annual_inc             42531 non-null float64
verification_status    42535 non-null object
loan_status            42535 non-null object
purpose                42535 non-null object
addr_state             42535 non-null object
total_acc              42506 non-null float64
total_pymnt            42535 non-null float64
application_type       42535 non-null object
dtypes: float64(6), object(10)
memory usage: 5.5+ MB


In [4]:
# Print the dimensions of data 
data.shape

(42535, 16)

## Generating a Hold Out Test Set

While we will be using K-fold cross validation to select an optimal model, we still want a final hold out test set that is completely independent of any modeling decisions. As such, pull out a sample of 30% of the total available data. For consistency of results, use random seed 42. 

In [5]:
# Features to build the model
features = ['loan_amnt', 'funded_amnt_inv', 'installment', 'annual_inc', 
            'home_ownership', 'verification_status', 'emp_length']

X = data[features]
y = data[['total_pymnt']]

# Split the data into training and test sets
X_train_intermediate, X_test, y_train_intermediate, y_test = train_test_split(X, y, test_size = .2, random_state = 42)
X_train, X_val, y_train, y_val = train_test_split(X_train_intermediate, y_train_intermediate, test_size=.3, random_state = 42)

In [6]:
print(len(X_train))
print(len(X_test))
print(len(X_val))
print(len(y_train))
print(len(y_test))
print(len(y_val))

23819
8507
10209
23819
8507
10209


In [7]:
X_train.head(3)

,loan_amnt,funded_amnt_inv,installment,annual_inc,home_ownership,verification_status,emp_length
24485,15000.0,14815.79564,382.29,100000.00,RENT,Source Verified,2 years
33734,25000.0,24886.91608,874.66,54383.42,RENT,Not Verified,5 years
23029,12000.0,11950.00000,254.91,57600.00,MORTGAGE,Source Verified,2 years


## Preprocessing (Numerical features) 

- Fill all missing values in numeric features with their respective means 
- Standardize all the numeric features  
- Convert the final results into DataFrames 

In [8]:
# Select continuous features
cont_features = ['loan_amnt', 'funded_amnt_inv', 'installment', 'annual_inc']

X_train_cont = X_train.loc[:, cont_features]
X_test_cont = X_test.loc[:, cont_features]

# Instantiate SimpleImputer - fill the missing values with the mean
si = SimpleImputer()

# Fit and transform the training data
X_train_imputed = si.fit_transform(X_train_cont)

# Transform test data
X_test_imputed = si.transform(X_test_cont)

# Instantiate StandardScaler
ss_X = StandardScaler()

# Fit and transform the training data
X_train_scaled = pd.DataFrame(ss_X.fit_transform(X_train_cont), columns = cont_features)

# Transform test data
X_test_scaled = pd.DataFrame(ss_X.transform(X_test_cont), columns = cont_features)

In [9]:
X_test_scaled

,loan_amnt,funded_amnt_inv,installment,annual_inc
0,-0.080399,0.044190,-0.342093,-0.466615
1,1.067453,1.085563,0.435240,-0.433533
2,0.392246,0.527833,-0.120389,-0.343637
3,0.122163,0.254469,0.377398,-0.331052
4,-0.958168,-0.867021,-0.964026,-0.329110
...,...,...,...,...
8502,-0.823127,-0.740853,-0.800742,-0.703220
8503,1.877702,2.076891,2.322966,-0.271721
8504,-0.823127,-0.726835,-0.705492,-0.487471
8505,0.122163,0.254469,0.230701,1.004798


## Preprocessing (Categorical features) 

- Fill all missing values in categorical features with the string `'missing'` 
- One-hot encode all categorical features 
- Convert the final results into DataFrames 


In [10]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23819 entries, 24485 to 2872
Data columns (total 7 columns):
loan_amnt              23819 non-null float64
funded_amnt_inv        23819 non-null float64
installment            23819 non-null float64
annual_inc             23816 non-null float64
home_ownership         23819 non-null object
verification_status    23819 non-null object
emp_length             23209 non-null object
dtypes: float64(4), object(3)
memory usage: 1.5+ MB


In [11]:
# Select only the categorical features
cat_features = ['home_ownership', 'verification_status', 'emp_length']
X_train_cat = X_train.loc[:, cat_features]
X_test_cat = X_test.loc[:, cat_features]

# Fill missing values with the string 'missing'
X_train_cat.replace(np.nan, 'missing', inplace=True)
X_test_cat.replace(np.nan, 'missing', inplace=True)

# OneHotEncode categorical variables
ohe = OneHotEncoder(sparse = False)

# Transform training and test sets
X_train_ohe = ohe.fit_transform(X_train_cat)
X_test_ohe = ohe.fit_transform(X_test_cat)

# Get all categorical feature names
#cat_columns = ohe.get_feature_names(input_features=X_train_ohe.columns)
cat_columns = ohe.get_feature_names(input_features=cat_features)

# Fit and transform the training data
X_train_categorical = pd.DataFrame(X_train_ohe, columns = cat_columns)

# Transform test data
X_test_categorical = pd.DataFrame(X_test_ohe, columns = cat_columns)


In [12]:
X_test_categorical.head(2)

,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,emp_length_1 year,emp_length_10+ years,emp_length_2 years,emp_length_3 years,emp_length_4 years,emp_length_5 years,emp_length_6 years,emp_length_7 years,emp_length_8 years,emp_length_9 years,emp_length_< 1 year,emp_length_missing
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [13]:
X_train_categorical.head(2)

,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,emp_length_1 year,emp_length_10+ years,emp_length_2 years,emp_length_3 years,emp_length_4 years,emp_length_5 years,emp_length_6 years,emp_length_7 years,emp_length_8 years,emp_length_9 years,emp_length_< 1 year,emp_length_missing
0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
X_train_categorical.shape

(23819, 20)

In [15]:
X_test_categorical.shape

(8507, 20)

In [16]:
type(X_train_ohe)

numpy.ndarray

In [17]:
ohe.get_feature_names()

array(['x0_MORTGAGE', 'x0_NONE', 'x0_OTHER', 'x0_OWN', 'x0_RENT',
       'x1_Not Verified', 'x1_Source Verified', 'x1_Verified',
       'x2_1 year', 'x2_10+ years', 'x2_2 years', 'x2_3 years',
       'x2_4 years', 'x2_5 years', 'x2_6 years', 'x2_7 years',
       'x2_8 years', 'x2_9 years', 'x2_< 1 year', 'x2_missing'],
      dtype=object)

Run the below cell to combine the numeric and categorical features. 

In [18]:
# Combine continuous and categorical feature DataFrames
X_train_all = pd.concat([X_train_scaled, X_train_categorical], axis=1)
X_test_all = pd.concat([X_test_scaled, X_test_categorical], axis=1)

# Number of input features
n_features = X_train_all.shape[1]

- Standardize the target DataFrames (`y_train` and `y_test`) 

In [19]:
# Instantiate StandardScaler
ss_y = StandardScaler()

# Fit and transform Y (train)
y_train_scaled = ss_y.fit_transform(y_train)

# Transform test Y (test)
y_test_scaled = ss_y.transform(y_test)

## Define a K-fold Cross Validation Methodology

Now that your have a complete holdout test set, you will perform k-fold cross-validation using the following steps: 

- Create a function that returns a compiled deep learning model 
- Use the wrapper function `KerasRegressor()` that defines how these folds are trained 
- Call the `cross_val_predict()` function to perform k-fold cross-validation 

In the cell below, we've defined a baseline model that returns a compiled Keras models. 

In [20]:
# Define a function that returns a compiled Keras model 
def create_baseline_model():
    
    # Initialize model
    model = models.Sequential()

    # First hidden layer
    model.add(layers.Dense(10, activation='relu', input_shape=(n_features,)))

    # Second hidden layer
    model.add(layers.Dense(5, activation='relu'))

    # Output layer
    model.add(layers.Dense(1, activation='linear'))

    # Compile the model
    model.compile(optimizer='SGD', 
                  loss='mse',  
                  metrics=['mse']) 
    
    # Return the compiled model
    return model

Wrap `create_baseline_model` inside a call to `KerasRegressor()`, and: 

- Train for 150 epochs 
- Set the batch size to 256 

> NOTE: Refer to the [documentation](https://keras.io/scikit-learn-api/) to learn about `KerasRegressor()`.  

In [21]:
# Wrap the above function for use in cross-validation
keras_wrapper_1 = KerasRegressor(build_fn=create_baseline_model(), epochs = 150, batch_size = 256)

Use `cross_val_predict()` to generate cross-validated predictions with: 
- 5-fold cv 
- scaled input (`X_train_all`) and output (`y_train_scaled`) 

In [22]:
# ⏰ This cell may take several mintes to run
# Generate cross-validated predictions
np.random.seed(123)
cv_baseline_preds = cross_val_predict(keras_wrapper_1, X_train_all, y_train_scaled, cv = 5)

TypeError: can't pickle _thread.lock objects

In [23]:
keras_wrapper_1.summary

AttributeError: 'KerasRegressor' object has no attribute 'summary'

In [24]:
# Define a function that returns a compiled Keras model 
def create_baseline_model_unwrapped(X_train, y_train, X_test, y_test):
    
    # Initialize model
    model = models.Sequential()

    # First hidden layer
    model.add(layers.Dense(10, activation='relu', input_shape=(n_features,)))

    # Second hidden layer
    model.add(layers.Dense(5, activation='relu'))

    # Output layer
    model.add(layers.Dense(1, activation='linear'))

    # Compile the model
    model.compile(optimizer='SGD', 
                  loss='mse',  
                  metrics=['mse']) 
    
    model.fit(X_train,
                y_train,
                epochs=120,
                batch_size=256,
                validation_data=(X_test, y_test))
    
    # Return the compiled model
    return model

- Find the RMSE on train data 

In [25]:
unwrapped_1 = create_baseline_model_unwrapped(X_train_all, y_train_scaled, X_test_all, y_test_scaled)

Train on 23819 samples, validate on 8507 samples
Epoch 1/120
23819/23819 [==============================] - 0s 14us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 2/120
23819/23819 [==============================] - 0s 8us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 3/120
23819/23819 [==============================] - 0s 7us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 4/120
23819/23819 [==============================] - 0s 8us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 5/120
23819/23819 [==============================] - 0s 8us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 6/120
23819/23819 [==============================] - 0s 7us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 7/120
23819/23

23819/23819 [==============================] - 0s 7us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 54/120
23819/23819 [==============================] - 0s 7us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 55/120
23819/23819 [==============================] - 0s 7us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 56/120
23819/23819 [==============================] - 0s 8us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 57/120
23819/23819 [==============================] - 0s 7us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 58/120
23819/23819 [==============================] - 0s 7us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 59/120
23819/23819 [==============================] - 0s 7us/step - los

23819/23819 [==============================] - 0s 10us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 106/120
23819/23819 [==============================] - 0s 8us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 107/120
23819/23819 [==============================] - 0s 7us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 108/120
23819/23819 [==============================] - 0s 9us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 109/120
23819/23819 [==============================] - 0s 9us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 110/120
23819/23819 [==============================] - 0s 8us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 111/120
23819/23819 [==============================] - 0s 7us/ste

In [ ]:
# RMSE on train data (scaled)


- Convert the scaled predictions back to original scale 
- Calculate RMSE in the original units with `y_train` and `baseline_preds` 

In [ ]:
# Convert the predictions back to original scale
baseline_preds = None

# RMSE on train data (original scale)


## Intentionally Overfitting a Model

Now that you've developed a baseline model, its time to intentionally overfit a model. To overfit a model, you can:
* Add layers
* Make the layers bigger
* Increase the number of training epochs

Again, be careful here. Think about the limitations of your resources, both in terms of your computers specs and how much time and patience you have to let the process run. Also keep in mind that you will then be regularizing these overfit models, meaning another round of experiments and more time and resources.

In [ ]:
# Define a function that returns a compiled Keras model 
def create_bigger_model():
    
    pass

In [ ]:
# Wrap the above function for use in cross-validation
keras_wrapper_2 = None

In [ ]:
# ⏰ This cell may take several mintes to run
# Generate cross-validated predictions
np.random.seed(123)
cv_bigger_model_preds = None

In [ ]:
# RMSE on train data (scaled)


## Regularizing the Model to Achieve Balance  

Now that you have a powerful model (albeit an overfit one), we can now increase the generalization of the model by using some of the regularization techniques we discussed. Some options you have to try include:  
* Adding dropout
* Adding L1/L2 regularization
* Altering the layer architecture (add or remove layers similar to above)  

This process will be constrained by time and resources. Be sure to test at least two different methodologies, such as dropout and L2 regularization. If you have the time, feel free to continue experimenting. 

In [ ]:
# Define a function that returns a compiled Keras model 
def create_regularized_model():
    
    pass

In [ ]:
# Wrap the above function for use in cross-validation
keras_wrapper_3 = None

In [ ]:
# ⏰ This cell may take several mintes to run
# Generate cross-validated predictions
np.random.seed(123)
cv_dropout_preds = None

In [ ]:
# RMSE on train data (scaled)


## Final Evaluation

Now that you have selected a network architecture, tested various regularization procedures and tuned hyperparameters via a validation methodology, it is time to evaluate your final model on the test set. Fit the model using all of the training data using the architecture and hyperparameters that were most effective in your experiments above. Afterwards, measure the overall performance on the hold-out test data which has been left untouched (and hasn't leaked any data into the modeling process)! 

In [ ]:
# ⏰ This cell may take several mintes to run

## Summary

In this lab, you investigated some data from *The Lending Club* in a complete data science pipeline to build neural networks with good performance. You began with reserving a hold-out set for testing which never was touched during the modeling phase. From there, you implemented a k-fold cross validation methodology in order to assess an initial baseline model and various regularization methods. 